In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir ('../')

In [ ]:
%pwd

In [ ]:
#entity return type it will retrun this variable 
#custome retun type
from dataclasses import dataclass

from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionconfig:
    root_dir: Path 
    source_urls: str
    local_data_files:Path
    unzip_dir:Path

In [ ]:
from textsummarzer.constants import *
from textsummarzer.utils.common import read_yaml,create_directory

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root]) 
    def get_data_ingestion_config(self)->DataIngestionconfig:
      
      config=self.config.data_ingestion
      create_directory([config.root_dir])
      data_ingestion_config=DataIngestionconfig(
      root_dir=config.root_dir ,
      source_urls=config.source_urls,
      local_data_files=config.local_data_files , #Path(config.local_data_files)
      unzip_dir=config.unzip_dir  #Path(config.unzip_dir)
      )
      return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
from textsummarzer.logging import logger
from textsummarzer.utils.common import get_size

class Data_Ingestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_urls,
                filename = self.config.local_data_files
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_files))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
#create the pipline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=Data_Ingestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

In [ ]:
import os
import urllib.request as request
from textsummarzer.logging import logger

source_url = "https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip"
local_data_file = "artifacts/data_ingestion/data.zip"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(local_data_file), exist_ok=True)

try:
    filename, header = request.urlretrieve(url=source_url, filename=local_data_file)
    logger.info(f"{filename} downloaded successfully with the following info:\n{header}")
    
    # Check if file exists and log its size
    if os.path.exists(local_data_file):
        logger.info(f"Downloaded file size: {os.path.getsize(local_data_file)} bytes")
    else:
        logger.error("File was not downloaded successfully.")
except Exception as e:
    logger.error(f"An error occurred during the download: {e}")
